In [1]:
import numpy as np
from integer_rref import i4mat_rref
# from sympy import Matrix

import pygad
from cytools import Polytope
from cytools.utils import find_new_affinely_independent_points


Info: A more recent version of CYTools is available: v1.0.4 -> v1.0.7.
We recommend upgrading before continuing.
On Linux and macOS you can update CYTools by running 'cytools --update'
and on Windows you can do this by running the updater tool.



In [2]:
from cytools import read_polytopes, fetch_polytopes

In [3]:
g = fetch_polytopes(limit=2)
p = next(g)
p = next(g)
print(p.vertices())
pts = p.points()[1:]

gene_size = pts.shape[0]

[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 2  4  5  0]
 [ 3  3  0  5]
 [-6 -8 -5 -5]]


In [4]:
def reduce_polytope(vertices):
    vertices_copy = np.array(vertices, copy=True)
    W = np.asarray(i4mat_rref(vertices.shape[0], vertices.shape[1], vertices_copy)[0]).astype(np.float64)
    local_vertices = np.round(vertices@np.linalg.pinv(W))
    idx = np.argwhere(np.all(local_vertices[..., :] == 0, axis=0))
    
    return Polytope(np.delete(local_vertices, idx, axis=1))

def generate_fitness_func(fibration_dimension):
    def fitness_func(ga_instance, solution, solution_idx):
        mask = (solution == 1)
        vertices = pts[mask]

        error = 0
        if vertices.size == 0:
            return -1000

        error += (np.linalg.matrix_rank(vertices) - fibration_dimension)**2
        p = reduce_polytope(vertices)

        # There probably exists a more efficient
        # way of doing this, but this works for now.
        error += (1 - p.is_reflexive())**2
        error += (fibration_dimension - p.dimension())**2

        return -error
    return fitness_func

In [9]:
# Initialize GA

# Search for K3 fibered CYs
fitness_func = generate_fitness_func(3)

ga_instance = pygad.GA(
    fitness_func=fitness_func,
    num_genes=gene_size,
    save_best_solutions=False,

    num_generations=1000,
    num_parents_mating=10,
    sol_per_pop=100,

    init_range_low=0,
    init_range_high=2,
    gene_type=int,

    parent_selection_type="rws",
    keep_parents=-1,
    crossover_type="two_points",
    mutation_type="random",
    mutation_percent_genes=20)

In [10]:
ga_instance.run()

In [11]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print(f"Found solution with fitness: {solution_fitness}")

Found solution with fitness: 0


In [12]:
reduce_polytope(pts[solution == 1]).points()

array([[ 0,  0,  0],
       [-2, -1, -2],
       [ 0,  1, -1],
       [ 1,  0,  1],
       [ 3,  0,  5],
       [ 2,  0,  3]])

In [13]:
reduce_polytope(pts[solution == 1])

A 3-dimensional reflexive lattice polytope in ZZ^3

In [ ]:
# TODO find triangulations of Polytope(pts) compatible with Polytope(pts[solution==1])